# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 8:30AM,246458,10,6966794,Emerginnova,Released
1,Sep 15 2022 8:30AM,246458,10,M-1-10582-1,Emerginnova,Released
2,Sep 15 2022 8:30AM,246460,10,MTR-22-0252,"Senseonics, Incorporated",Released
3,Sep 15 2022 8:30AM,246462,10,TXMD-DR-0100-BO,"TherapeuticsMD, Inc.",Released
4,Sep 14 2022 2:43PM,246447,20,ALU0007930,Alumier Labs Inc.,Released
5,Sep 14 2022 2:43PM,246447,20,ALUR809831162,Alumier Labs Inc.,Released
6,Sep 14 2022 2:43PM,246447,20,ALUR842988174,Alumier Labs Inc.,Released
7,Sep 14 2022 2:41PM,246444,19,ACG-2102491252,ACG North America LLC,Released
8,Sep 14 2022 2:41PM,246444,19,ACG-210491253,ACG North America LLC,Released
9,Sep 14 2022 1:56PM,246442,19,60017402,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,246444,Released,2
17,246447,Released,3
18,246458,Released,2
19,246460,Released,1
20,246462,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246444,NaN,NaN,2.0
246447,NaN,NaN,3.0
246458,NaN,NaN,2.0
246460,NaN,NaN,1.0
246462,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246356,0.0,0.0,1.0
246357,0.0,0.0,1.0
246358,0.0,1.0,0.0
246361,0.0,0.0,1.0
246362,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246356,0,0,1
246357,0,0,1
246358,0,1,0
246361,0,0,1
246362,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246356,0,0,1
1,246357,0,0,1
2,246358,0,1,0
3,246361,0,0,1
4,246362,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246356,,,1
1,246357,,,1
2,246358,,1,
3,246361,,,1
4,246362,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 8:30AM,246458,10,Emerginnova
2,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated"
3,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc."
4,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.
7,Sep 14 2022 2:41PM,246444,19,ACG North America LLC
9,Sep 14 2022 1:56PM,246442,19,"GUSA Granules USA, Inc."
10,Sep 14 2022 1:43PM,246441,19,"GUSA Granules USA, Inc."
11,Sep 14 2022 1:32PM,246439,19,"GUSA Granules USA, Inc."
12,Sep 14 2022 1:23PM,246435,20,"ACI Healthcare USA, Inc."
41,Sep 14 2022 12:38PM,246429,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 15 2022 8:30AM,246458,10,Emerginnova,,,2
1,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",,,1
2,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",,,1
3,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,,,3
4,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,,,2
5,Sep 14 2022 1:56PM,246442,19,"GUSA Granules USA, Inc.",,,1
6,Sep 14 2022 1:43PM,246441,19,"GUSA Granules USA, Inc.",,,1
7,Sep 14 2022 1:32PM,246439,19,"GUSA Granules USA, Inc.",,,1
8,Sep 14 2022 1:23PM,246435,20,"ACI Healthcare USA, Inc.",6,14,9
9,Sep 14 2022 12:38PM,246429,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246458,10,Emerginnova,2,,
1,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1,,
2,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1,,
3,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3,,
4,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,2,,
5,Sep 14 2022 1:56PM,246442,19,"GUSA Granules USA, Inc.",1,,
6,Sep 14 2022 1:43PM,246441,19,"GUSA Granules USA, Inc.",1,,
7,Sep 14 2022 1:32PM,246439,19,"GUSA Granules USA, Inc.",1,,
8,Sep 14 2022 1:23PM,246435,20,"ACI Healthcare USA, Inc.",9,14,6
9,Sep 14 2022 12:38PM,246429,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246458,10,Emerginnova,2,,
1,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1,,
2,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1,,
3,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3,,
4,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246458,10,Emerginnova,2.0,NaN,NaN
1,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1.0,NaN,NaN
3,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3.0,NaN,NaN
4,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 8:30AM,246458,10,Emerginnova,2.0,0.0,0.0
1,Sep 15 2022 8:30AM,246460,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Sep 15 2022 8:30AM,246462,10,"TherapeuticsMD, Inc.",1.0,0.0,0.0
3,Sep 14 2022 2:43PM,246447,20,Alumier Labs Inc.,3.0,0.0,0.0
4,Sep 14 2022 2:41PM,246444,19,ACG North America LLC,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1478595,6.0,1.0,0.0
102,492713,2.0,0.0,0.0
17,492723,2.0,0.0,0.0
19,1232190,6.0,2.0,0.0
20,739302,12.0,15.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1478595,6.0,1.0,0.0
1,102,492713,2.0,0.0,0.0
2,17,492723,2.0,0.0,0.0
3,19,1232190,6.0,2.0,0.0
4,20,739302,12.0,15.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,6.0,1.0,0.0
1,102,2.0,0.0,0.0
2,17,2.0,0.0,0.0
3,19,6.0,2.0,0.0
4,20,12.0,15.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,6.0
1,102,Released,2.0
2,17,Released,2.0
3,19,Released,6.0
4,20,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,17,19,20
Status,,,,,
Completed,0.0,0.0,0.0,0.0,6.0
Executing,1.0,0.0,0.0,2.0,15.0
Released,6.0,2.0,2.0,6.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,17,19,20
0,Completed,0.0,0.0,0.0,0.0,6.0
1,Executing,1.0,0.0,0.0,2.0,15.0
2,Released,6.0,2.0,2.0,6.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,17,19,20
0,Completed,0.0,0.0,0.0,0.0,6.0
1,Executing,1.0,0.0,0.0,2.0,15.0
2,Released,6.0,2.0,2.0,6.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()